In [36]:
import pandas as pd

names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
DayAheadPrices=pd.read_csv("Daten entsoe/energy-charts_Stromproduktion_und_Börsenstrompreise_in_Deutschland_2024.csv",skiprows=2,names=names)

In [37]:
#preis_menge=DayAheadPrices
DayAheadPrices['datum']=pd.to_datetime(DayAheadPrices['Datum(UTC)'])
DayAheadPrices=DayAheadPrices.set_index('datum')
preis_menge=DayAheadPrices

In [38]:
names=['Datum','Jahr', 'Monat', 'Tag','PV IK8']
IKPV=pd.read_csv("Daten IK8/2024 PV Daily.csv",skiprows=1,names=names,sep=';',decimal=',',parse_dates=['Datum'],date_format='%d.%m.%Y').fillna(0)
IKPV=IKPV.drop(columns=['Jahr', 'Monat', 'Tag'])
IKPV=IKPV.set_index('Datum')

In [39]:
IKStrom=pd.read_csv("Daten IK8/2024 Hist Strom.csv",date_format='ISO8601',parse_dates=True,decimal='.')
IKStrom['last_changed dt']=pd.to_datetime(IKStrom['last_changed']).dt.floor('h')
IKStrom=IKStrom.set_index('last_changed dt')
ent_filter=['sensor.haushalt_positive_active_energy_total','sensor.haushalt_negative_active_energy_total','sensor.waermepumpe_positive_active_energy_total','sensor.waermepumpe_negative_active_energy_total']
IKStrom=IKStrom[IKStrom['entity_id'].isin(ent_filter)]
IKStrom['state']=pd.to_numeric(IKStrom['state'])
IKStrom=IKStrom.groupby('entity_id').resample('h')['state'].agg(['min']).ffill()
IKStrom=IKStrom.reset_index().pivot(index=['last_changed dt'],columns='entity_id',values='min')
IKStrom=IKStrom.ffill()

In [40]:
preis_menge[['Haus H','Haus S','WP H','WP S']]=IKStrom.diff().shift(-1).fillna(0)
#preis_menge['WP PV']=preis_menge['Haus H']-preis_menge['WP H']


In [182]:
cnt = 0
gap_cnt=0
gap=[]
cols=['WP S','WP H']
names=['Uhrzeit','kW']
#cols=['Haus S','Haus H']
#gap_save=[]
for index,row in preis_menge.iterrows():
    sv=""
    #sm = row['Haus S']+row['Haus H']
    sm = row[cols[0]]+row[cols[1]]
    if sm == 0:
        #print(index)
        cnt +=1
        gap_cnt += 1
        gap=gap+[index]
    else:
        if len(gap)>0:
            #print(index,gap_cnt,cnt)
            day_before_gap=min(gap).floor('d')-pd.DateOffset(days=1)
            filler=preis_menge[preis_menge.index.dayofyear==day_before_gap.dayofyear]

            gap_df=pd.DataFrame(gap,columns=['Datum'])
            gap_days=pd.DataFrame(gap_df['Datum'].dt.date.unique(),columns=['Datum']).set_index('Datum')
            gap_days['PV IK8']=IKPV['PV IK8']
            
            # Wieviele % und wie viel PW muss insgesamt auf den Tag verteilt werden

            #li_pro_kw=[]
            for gd_index,gd_row in gap_days.iterrows():
                #print()
                r=gd_index
                miss_IK_PV=pd.read_csv("Daten IK8/Wechselrichter_Tag_{}-{:02d}-{:02d}.csv".format(r.year,r.month,r.day),names=names,sep=';',decimal=',',skiprows=1)
            
            for fill_up in gap:
                for app in cols:
                    None
                    pct=filler[filler.index.hour==fill_up.hour][app]/sum(filler[app])
            
            save_gap=gap
            save_row=row
            gap=[]
            gap_cnt=0
            #print(row)
            #print(gap_days)
print(cnt)

421


In [146]:
gap_days=pd.DataFrame(gap_df['Datum'].dt.date.unique(),columns=['Datum']).set_index('Datum')

In [183]:
miss_IK_PV

,Uhrzeit,kW
0,00:15,0.0
1,00:30,0.0
2,00:45,0.0
3,01:00,0.0
4,01:15,0.0
...,...,...
91,23:00,0.0
92,23:15,0.0
93,23:30,0.0
94,23:45,0.0


In [176]:
"Daten IK8/Wechselrichter_Tag_{}-{:02d}-{:02d}.csv".format(r.year,r.month,r.day)

'Daten IK8/Wechselrichter_Tag_2024-05-17.csv'

In [91]:
filler[filler.index.hour==14]['Haus H']/sum(filler['Haus H'])

datum
2024-12-28 14:00:00+00:00    0.014287
Name: Haus H, dtype: float64

In [117]:
import timestamp

ModuleNotFoundError: No module named 'timestamp'

In [116]:
dir(pd.Timestamp)

['__add__',
 '__array_priority__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__radd__',
 '__reduce__',
 '__reduce_cython__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__setstate__',
 '__setstate_cython__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_creso',
 '_date_repr',
 '_from_dt64',
 '_from_value_and_reso',
 '_repr_base',
 '_round',
 '_time_repr',
 '_value',
 'as_unit',
 'asm8',
 'astimezone',
 'ceil',
 'combine',
 'ctime',
 'date',
 'day',
 'day_name',
 'day_of_week',
 'day_of_year',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'dst',
 'floor',
 'fold',
 'fromisocalendar',
 'fromisoformat',
 'fromordinal',
 'fromtimestamp',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_mo